# 04 Risk and Factor Analysis
Evaluate regime LS and rotation strategies against Fama-French factors.

- **Goal:** check if strategy returns are explained by common equity factors.
- **Alpha:** intercept in excess-return regression; annualized.
- **Betas:** exposure to market (MKT-RF), size (SMB), value (HML), profitability (RMW), investment (CMA).
- Interpretation: Is alpha significant? Do betas look like a tilt or distinct source?

In [ ]:
import sys
from pathlib import Path

import numpy as np
import pandas as pd

# Ensure repo root on sys.path
repo_root = Path.cwd().resolve()
if not (repo_root / 'src').exists():
    repo_root = repo_root.parent
if str(repo_root) not in sys.path:
    sys.path.append(str(repo_root))

from src.data.ff_factors import load_ff_factors_monthly
from src.data.macro_loader import load_tnx_10y, load_vix
from src.analysis.factor_analysis import align_strategy_and_factors, run_ff_regression
from src.analysis.metrics import compute_cagr, compute_annual_vol, compute_sharpe, compute_max_drawdown
from src.data.etf_loader import load_clean_prices
from src.signals.ls_biotech_pharma import build_monthly_ls_weights
from src.signals.regime import compute_monthly_features, classify_regime
from src.signals.rotation_signals import build_monthly_rotation_weights
from src.backtest.engine import run_backtest

%matplotlib inline


### Load strategy returns (re-run quickly here for demonstration)
In practice, cache daily returns from prior notebooks and load them.

In [ ]:
prices = load_clean_prices()
idx = prices.index
tnx_yield = load_tnx_10y(start=idx.min().strftime('%Y-%m-%d'), end=idx.max().strftime('%Y-%m-%d')).reindex(idx).ffill()
spy_prices = prices['SPY']
vix = load_vix(start=idx.min().strftime('%Y-%m-%d'), end=idx.max().strftime('%Y-%m-%d')).reindex(idx).ffill()
features = compute_monthly_features(tnx_yield, spy_prices, vix)
regime_labels = classify_regime(features)
ls_weights = build_monthly_ls_weights(regime_labels, prices.index)
ls_result = run_backtest(prices[['XBI','XPH']], ls_weights)

rotation_prices = prices[['XBI','XPH','IHF','IHI','XLV']]
rotation_weights = build_monthly_rotation_weights(rotation_prices)
rotation_result = run_backtest(rotation_prices, rotation_weights)


### Load Fama-French factors
Assumes processed CSV at `data_processed/ff_factors_monthly.csv`.

In [ ]:
ff = load_ff_factors_monthly()
ff.head()


### Helper to run regression and print results

In [ ]:
def run_and_report(label, daily_returns):
    strat_excess, ff_aligned = align_strategy_and_factors(daily_returns, ff)
    reg = run_ff_regression(strat_excess, ff_aligned)
    print(f"\n{label} vs FF5 (monthly):")
    print(f"Alpha (ann): {reg['alpha_annual']:.2%} (t={reg['alpha_tstat']:.2f})")
    print(f"R^2: {reg['r2']:.3f}, N: {reg['n_obs']}")
    for fac, beta in reg['betas'].items():
        tstat = reg['betas_tstat'][fac]
        print(f"  {fac}: beta={beta:.3f}, t={tstat:.2f}")
    return reg


In [ ]:
reg_ls = run_and_report('Regime LS', ls_result.daily_returns)
reg_rot = run_and_report('Rotation', rotation_result.daily_returns)


### Interpretation (to refine with real data)
- Is alpha significant? Check t-stat near or above ~2 in absolute value.
- Betas indicate factor tilts: market, size, value, profitability, investment.
- High R^2 and large betas suggest a factor-tilt strategy; low R^2 with meaningful alpha hints at distinct timing/exposure.